In [1]:
import sagemaker, boto3

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/cheekeanchan/Library/Application Support/sagemaker/config.yaml


In [2]:
TARGET_IMG = 'ml-infra-pipelines-train-dev'
OUTPUT_PATH = "s3://ml-infra-pipelines-dev/output/"
role = 'ml-infra-pipelines-dev-sagemaker'
file_location = "s3://ml-infra-pipelines-dev/sample/training/iris.csv"

sess = sagemaker.Session()
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name
image_uri = f'{account}.dkr.ecr.{region}.amazonaws.com/{TARGET_IMG}:latest'
model = sagemaker.estimator.Estimator(
    image_uri,
    role,
    instance_count=1,
    instance_type='ml.m4.xlarge',
    output_path=OUTPUT_PATH,
    sagemaker_session=sess,
    enable_cloudwatch_metrics=True,
)
status = 'succeed'
model.fit(file_location)

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/cheekeanchan/Library/Application Support/sagemaker/config.yaml


INFO:sagemaker:Creating training-job with name: ml-infra-pipelines-train-dev-2023-11-13-04-46-12-225


2023-11-13 04:46:14 Starting - Starting the training job...
2023-11-13 04:46:39 Starting - Preparing the instances for training......
2023-11-13 04:47:34 Downloading - Downloading input data...
2023-11-13 04:48:15 Training - Training image download completed. Training in progress.
2023-11-13 04:48:15 Uploading - Uploading generated training model
2023-11-13 04:48:15 Completed - Training job completed
..Training seconds: 42
Billable seconds: 42


In [4]:
import subprocess
from time import gmtime, strftime
bashCommand = "tar -cvpzf model.tar.gz ./temp/decision-tree-model.pkl ./train/custom_image/inference.py"
process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
output, error = process.communicate()

a ./temp/decision-tree-model.pkl
a ./train/custom_image/inference.py


In [26]:
model_name = 'ml-infra-iris-predictor'
model_package_arn = "arn:aws:sagemaker:ap-southeast-1:852288348919:model-package/ml-infra-iris-predictor/6"
role = 'ml-infra-pipelines-dev-sagemaker'
instance_type = "ml.c5.xlarge"
instance_count = 1
blue_green_deploy = False

In [15]:
boto_session = boto3.session.Session()
s3 = boto_session.resource('s3')
default_bucket = "ml-infra-pipelines-dev"
filename = 'output/temp/model.tar.gz'

response = s3.meta.client.upload_file('model.tar.gz', default_bucket, filename)
model_artifacts = f"s3://{default_bucket}/{filename}"

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


In [33]:
client = boto3.client('sagemaker')
iam_client = boto3.client('iam')
role_arn = iam_client.get_role(RoleName=role)['Role']['Arn']
# client.delete_endpoint(EndpointName=model_name)

image_uri = sagemaker.image_uris.retrieve(
    framework="sklearn",
    region=region,
    version="0.23-1",
    py_version="py3",
    instance_type="ml.m5.xlarge",
)

model_name = 'ml-infra-iris-predictor'
new_config_name = model_name + '-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

model_response = client.create_model(
    ModelName=new_config_name,
    Containers=[
        {
            "Image": image_uri,
            "Mode": "SingleModel",
            "ModelDataUrl": model_artifacts,
            "Environment": {'SAGEMAKER_SUBMIT_DIRECTORY': model_artifacts,
                           'SAGEMAKER_PROGRAM': 'inference.py'} 
        }
    ],
    ExecutionRoleArn=role_arn,
)


In [34]:
### endpoint config creation ###
print("Model arn : {}".format(model_response["ModelArn"]))
endpoint_config_response = client.create_endpoint_config(
    EndpointConfigName=new_config_name,
    ProductionVariants=[{
        'InstanceType': instance_type,
        'InitialInstanceCount': instance_count,
        'InitialVariantWeight': 1,
        'ModelName': new_config_name,
        'VariantName': 'sklearnvariant'
        }]
)

# blue green deployment
if blue_green_deploy:
    deploy_config = {
        "BlueGreenUpdatePolicy": {
            "TrafficRoutingConfiguration": {
                "Type": "LINEAR",
                "LinearStepSize": {
                    "Type": "CAPACITY_PERCENT",
                    "Value": 20
                    },
                "WaitIntervalInSeconds": 300
                },
            "TerminationWaitInSeconds": 300,
            "MaximumExecutionTimeoutInSeconds": 3600
            }
        }
else:
    deploy_config = None

### endpoint creation ###
try:
    endpoint_response = client.describe_endpoint(EndpointName=model_name)
    client.delete_endpoint(EndpointName=model_name)
    print("Endpoint Exists, updating...")
    endpoint_exists = True
except Exception as e:
    print(f"Error: {e}. Creating New Endpoint")
    endpoint_exists = False

# if endpoint_exists:
#     endpoint_response = client.update_endpoint(
#         EndpointName=model_name,
#         EndpointConfigName=new_config_name,
#         DeploymentConfig=deploy_config
#     )
# else:
endpoint_create_response = client.create_endpoint(
    EndpointName=model_name,
    EndpointConfigName=new_config_name
)

# printing creation status
print("Creating Endpoint")
while True:
    endpoint_response = client.describe_endpoint(EndpointName=model_name)
    if endpoint_response["EndpointStatus"] != "Creating":
        print(endpoint_response["EndpointStatus"])
        print(f"Endpoint Arn: {endpoint_response['EndpointArn']}")
        break
    time.sleep(15)

Model arn : arn:aws:sagemaker:ap-southeast-1:852288348919:model/ml-infra-iris-predictor-2023-11-13-05-15-46
Error: An error occurred (ValidationException) when calling the DescribeEndpoint operation: Could not find endpoint "ml-infra-iris-predictor".. Creating New Endpoint
Creating Endpoint


NameError: name 'time' is not defined

In [36]:
print("Creating Endpoint")
while True:
    endpoint_response = client.describe_endpoint(EndpointName=model_name)
    if endpoint_response["EndpointStatus"] != "Creating":
        print(endpoint_response["EndpointStatus"])
        print(f"Endpoint Arn: {endpoint_response['EndpointArn']}")
        break
    time.sleep(15)

Creating Endpoint
Failed
Endpoint Arn: arn:aws:sagemaker:ap-southeast-1:852288348919:endpoint/ml-infra-iris-predictor


In [38]:
client.delete_endpoint(EndpointName=model_name)

{'ResponseMetadata': {'RequestId': 'da14e6a3-3e54-4068-ad03-26c48c04b167',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'da14e6a3-3e54-4068-ad03-26c48c04b167',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Mon, 13 Nov 2023 06:02:48 GMT'},
  'RetryAttempts': 0}}